# Voxel Visual Odometry Integrated Pipeline
## Importing requirements and scripts

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from collections import deque

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from helperScripts.TimeKeeper import TimeKeeper
from VoxelGrid import VoxelGrid

%matplotlib ipympl

## Load calibrations and other data

In [2]:
imageProcessor = ImageProcessor()
imageProcessor.verbose = True
imageProcessor.loadMonoCalibration()
imageProcessor.loadStereoCalibration()
imageProcessor.loadCameraProperties()
imageProcessor.loadStereoRectify()
imageProcessor.initUndistortRectifyMap()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/cameraProperties.json
Loaded camera properties
Reading from data/stereoRectify.json
Loaded stereo rectification data


## Create stereo matcher

In [3]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

Reading from data/parametersSGBM.json


## Create voxel grid

In [4]:
voxelGrid = VoxelGrid(stereoMatcher=stereoMatcher, imageProcessor=imageProcessor)
voxelGrid.verbose = True

## TimeKeeper for performance metrics

In [5]:
timeKeeper = TimeKeeper()

## Loading images

In [6]:
path = "testImages/visualOdometryTestImages"
folderChoice = 2

path = "".join([path, "/", str(folderChoice)])

imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))

if not len(imageGlobL)==len(imageGlobR):
    print("Images could not be matched")

print ("Selections: 0-{}".format(len(imageGlobL)-2))

Selections: 0-13


In [7]:
# Top images
imagesL = []
for imageFile in imageGlobL:
    imagesL.append(cv2.imread(imageFile))

# Bottom images
imagesR = []
for imageFile in imageGlobR:
    imagesR.append(cv2.imread(imageFile))

print(len(imagesL), len(imagesR))

15 15


### View image pair

In [8]:
selection = 0

plt.figure(figsize=(7,7))
plt.suptitle("Image pair")
plt.imshow(cv2.cvtColor(cv2.rotate(\
                    np.hstack([imagesL[selection], imagesR[selection]]), \
                cv2.ROTATE_90_CLOCKWISE), \
            cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.figure(figsize=(7,7))
plt.suptitle("Sequential image pair")
plt.imshow(cv2.cvtColor(cv2.rotate(\
                    np.hstack([imagesL[selection], imagesL[selection+1]]), \
                cv2.ROTATE_90_CLOCKWISE), \
            cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Visual Odometry
## Feature extraction
### Initializing feature extractor

In [10]:
# Extraction function
orb = cv2.ORB_create()

def extractFeatures(image, orb=orb):
    """Find keypoints and descriptors for the image"""
    keypoints = orb.detect(image, None)
    keypoints, descriptors = orb.compute(image, keypoints)
    
    return keypoints, descriptors

keypoints0, descriptors0 = extractFeatures(imagesL[selection])
keypoints1, descriptors1 = extractFeatures(imagesL[selection+1])

print("Number of features detected in frame {}: {}"\
                                .format(selection, len(keypoints0)))
print("Coordinates of first keypoint in frame {}: {}"\
                                .format(selection, str(keypoints0[0].pt)))

Number of features detected in frame 0: 500
Coordinates of first keypoint in frame 0: (288.0, 130.0)


### Visualize extracted features

In [11]:
def visualizeFeatures(image, keypoints, flag):
    """Visualize extracted features in image"""
    display = cv2.drawKeypoints(image, keypoints, None, flags=flag)
    plt.figure(figsize=(7, 5))
    plt.imshow(cv2.cvtColor\
                (cv2.rotate(\
                    display, cv2.ROTATE_90_CLOCKWISE), \
                cv2.COLOR_BGR2RGB))

visualizeFeatures(imagesL[selection], keypoints0, 4)
visualizeFeatures(imagesL[selection], keypoints0, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Initialize feature matcher

In [12]:
bfMatcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

def matchFeatures(descriptors0, descriptors1, \
                                bfMatcher=bfMatcher, bestNMatches=100):
    """Match features from two images"""
    match = bfMatcher.match(descriptors0, descriptors1)
    match = sorted(match, key = lambda x:x.distance)

    return match[:bestNMatches]

match01 = matchFeatures(descriptors0, descriptors1, bfMatcher)

print("Number of features matched in frames {} and {}: {}"\
                        .format(selection, selection+1, len(match01)))

Number of features matched in frames 0 and 1: 100


### Visualize feature match

In [13]:
def visualizeMatches(image0, keypoints0, image1, keypoints1, match):
    imageMatches = cv2.drawMatches(image0, keypoints0, \
                            image1, keypoints1, match, None, flags=2)

    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(cv2.rotate(\
                    imageMatches, cv2.ROTATE_90_CLOCKWISE), \
                cv2.COLOR_BGR2RGB))

visualizeMatches(imagesL[selection], keypoints0, \
                imagesL[selection+1], keypoints1, match01)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Trajectory Estimation

In [14]:
def estimateMotion(match, keypoints0, keypoints1, k=imageProcessor.cameraMatrixL):
    """Estimate camera motion from a pair of subsequent image frames"""
    imagePoints0 = []
    imagePoints1 = []

    for m in match:
        train_idx = m.trainIdx
        query_idx = m.queryIdx

        p1x, p1y = keypoints0[query_idx].pt 
        imagePoints0.append([p1x,p1y])

        p2x,p2y = keypoints1[train_idx].pt 
        imagePoints1.append([p2x,p2y])
    
    E, mask = cv2.findEssentialMat(\
                    np.array(imagePoints0), np.array(imagePoints1), k, \
                    cv2.RANSAC, 0.999, 1.0) 

    retval, rmat, tvec, mask = cv2.recoverPose(E, np.array(imagePoints0), \
                    np.array(imagePoints1), k)

    return rmat, tvec, imagePoints0, imagePoints1

rmat, tvec, imagePoints0, imagePoints1 = estimateMotion(match01, \
                        keypoints0, keypoints1)

print("Estimated rotation:\n {0}".format(rmat))
print("Estimated translation:\n {0}".format(tvec))

Estimated rotation:
 [[ 0.99924559  0.03707732 -0.01155537]
 [-0.03609422  0.99645086  0.07604537]
 [ 0.01433391 -0.07557091  0.9970374 ]]
Estimated translation:
 [[ 0.04634577]
 [ 0.08936792]
 [-0.99491982]]


## Movement visualization

In [15]:
def visualizeCameraMovement(image0, imagePoints0, \
                image1, imagePoints1, showImageAfterMove=False):
    """Visualize camera movement across frames"""
    image0 = image0.copy()
    image1 = image1.copy()

    for i in range(0, len(imagePoints0)):
        # Coordinates of a point on t frame
        p1 = (int(imagePoints0[i][0]), int(imagePoints0[i][1]))
        # Coordinates of the same point on t+1 frame
        p2 = (int(imagePoints1[i][0]), int(imagePoints1[i][1]))

        cv2.circle(image0, p1, 5, (0, 255, 0), 1)
        cv2.arrowedLine(image0, p1, p2, (0, 255, 0), 1)
        cv2.circle(image0, p2, 5, (255, 0, 0), 1)

        if showImageAfterMove:
            cv2.circle(image1, p2, 5, (255, 0, 0), 1)
    
    if showImageAfterMove: 
        return image1
    else:
        return image0

imageMovementBefore = visualizeCameraMovement(imagesL[selection], \
                    imagePoints0, imagesL[selection+1], imagePoints1)

imageMovementAfter = visualizeCameraMovement(imagesL[selection], \
                    imagePoints0, imagesL[selection+1], imagePoints1, \
                    showImageAfterMove=True)

plt.figure(figsize=(7,7))
plt.imshow(cv2.cvtColor(cv2.rotate(\
                np.hstack([imageMovementBefore, imageMovementAfter]), \
                    cv2.ROTATE_90_CLOCKWISE), \
                cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Voxel grid
## Creating disparity map
### Convert to grayscale and undistort

In [16]:
imageProcessor.convertToGrayscale(imagesL[selection], imagesR[selection])
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

fig = plt.figure(figsize=(7,7))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(cv2.rotate(\
            np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), \
            cv2.ROTATE_90_CLOCKWISE), cv2.COLOR_BGR2RGB))

fig = plt.figure(figsize=(7,7))
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(cv2.rotate(\
        imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), \
        cv2.ROTATE_90_CLOCKWISE), cv2.COLOR_BGR2RGB))

fig = plt.figure(figsize=(7,12))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [17]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE))

minDisparity: 9.0
maxDisparity: 41.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map (not necessary)

In [18]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

9.0
41.0
(640, 360)
481.33795
2192.7617


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Compute voxel grid
### Compute, filter, and rotate point cloud

In [19]:
pointSubsample = 24
voxelSize = 100
voxelStopFraction = 10
occupancyThreshold = 10

def generatePointCloud(disparityMapL, dispToDepthMatrix):
    points = cv2.reprojectImageTo3D(\
            disparityMapL, \
            dispToDepthMatrix)

    # Reshaping to a list of 3D coordinates
    pointCloud = points.reshape(\
                (points.shape[0]*points.shape[1],3))[0::pointSubsample]\
                                                .astype(np.int16)

    return pointCloud

pointCloud = generatePointCloud(stereoMatcher.disparityMapL,\
                                imageProcessor.dispToDepthMatrix)

print("Points in unfiltered pointcloud: {}".format(pointCloud.shape[0]))

def filterPointCloud(pointCloud):
    # Filtering x values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 0]>pointCloud[:, 0].min(), \
            pointCloud[:, 0]<pointCloud[:, 0].max())]
    # Filtering y values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 1]>pointCloud[:, 1].min(), \
            pointCloud[:, 1]<pointCloud[:, 1].max())]
    # Filtering z values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 2]>pointCloud[:, 2].min(), \
            pointCloud[:, 2]<pointCloud[:, 2].max())]

    return pointCloud

pointCloud = filterPointCloud(pointCloud)

print("Points in filtered pointcloud: {}".format(pointCloud.shape[0]))

redefineRotationMatrix = np.array([ [ 0,  0, -1],
                                    [ 0,  1,  0],
                                    [ 1,  0,  0] ])

def rotateGrid(grid, rotationMatrix):
    """Rotate point cloud or voxel grid with given rotation matrix"""
    grid = np.dot(grid[:], rotationMatrix)

    return grid

def plotGrid(grid, s):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111, projection = "3d")

    ax.scatter(grid[:,0], grid[:,1], grid[:,2], s=s)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_zlabel("$z$")

    # Camera axis begins at x=0 and looks to positive x
    ax.set_xlim(0,2000)
    ax.set_ylim(-1000,1000)
    ax.set_zlim(-1000,1000)

    plt.show()

plotGrid(rotateGrid(pointCloud, redefineRotationMatrix), 1)

Points in unfiltered pointcloud: 9600
Points in filtered pointcloud: 6214


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [20]:
baseVoxelGrid = None

def voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction):
    """Create voxelized representation of given point cloud"""
    iterations = 0
    newVoxelGrid = []
    initialSize = pointCloud.shape[0]
    remainingPoints = initialSize
    samplingLimit = np.zeros_like(pointCloud[0])
        
    while remainingPoints>(initialSize/voxelStopFraction):

        sampledPoint = pointCloud[np.random.randint(0,remainingPoints)]

        for n in range(3):
            samplingLimit[n]=\
                (sampledPoint[n]//voxelSize)*voxelSize

        mask = np.ones(remainingPoints, dtype=bool)

        for n in range(len(sampledPoint)):
            mask = np.logical_and(mask, np.logical_and(\
                pointCloud[:,n]>=samplingLimit[n], \
                pointCloud[:,n]<samplingLimit[n]+voxelSize))

        pointsInVoxel = pointCloud[mask]

        if len(pointsInVoxel)>occupancyThreshold:
            voxelMidpoint = samplingLimit+voxelSize/2
            newVoxelGrid.append(voxelMidpoint)

        pointCloud = pointCloud[np.invert(mask)]

        iterations+=1

        remainingPoints = pointCloud.shape[0]

    newVoxelGrid = np.array(newVoxelGrid, dtype=np.int16)

    return newVoxelGrid, iterations

baseVoxelGrid, iterations = voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction)

baseVoxelGrid = rotateGrid(baseVoxelGrid, redefineRotationMatrix)

print("".join(["Voxels in grid: {}; {} iterations"]).format(\
                baseVoxelGrid.shape[0], iterations))

plotGrid(baseVoxelGrid, voxelSize)

Voxels in grid: 56; 71 iterations


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Refine voxel grid
### Fetching camera fov

In [21]:
# Horizontal field of view (degrees)
fovH = ((imageProcessor.fovYL+imageProcessor.fovYR)/4)*np.pi/180
fovH -= fovH/8
print("Horizontal:", fovH*180/np.pi)

# Vertical field of view (degrees)
fovV = ((imageProcessor.fovXL+imageProcessor.fovXR)/4)*np.pi/180
fovV -= fovV/8
print("Vertical:", fovV*180/np.pi)

Horizontal: 24.915932084055157
Vertical: 14.867033874651561


### Get voxels near camera

In [22]:
def returnVoxelsInRange(voxelGrid, cameraPosition, distance=1500):
    """Return voxels that are within given distance from cameraPosition 
    in voxelGrid, along with yaw and distance"""
    translatedVoxels = voxelGrid-cameraPosition
    distanceToVoxels = np.linalg.norm(translatedVoxels, axis=1)

    voxelsInRange = voxelGrid[distanceToVoxels<=distance]
    translatedVoxelsInRange = voxelsInRange - cameraPosition

    distanceToVoxelsInRange = distanceToVoxels[distanceToVoxels<=distance]
    yawToVoxelsInRange = np.arctan2(translatedVoxelsInRange[:,1], \
                                        translatedVoxelsInRange[:,0])

    return voxelsInRange, yawToVoxelsInRange, distanceToVoxelsInRange

### Get yaw of camera

In [23]:
def getCameraYawRange(rotationMatrix, currentRotationMatrix=None):
    """Return camera yaw due to rotationMatrix on currentRotationMatrix"""
    cameraDirectionVector = np.array([0,0,100])

    cameraDirectionVector = np.dot(cameraDirectionVector, rotationMatrix)

    cameraYaw = np.arctan2(cameraDirectionVector[1], cameraDirectionVector[0])

    cameraYawRange = np.array([cameraYaw+fovH, cameraYaw-fovH])
    # Wrapping around values at -180, 180 degrees
    for n in range(len(cameraYawRange)):
        if cameraYawRange[n]>np.pi:
            cameraYawRange[n] -= 2*np.pi
        if cameraYawRange[n]<=-np.pi:
            cameraYawRange[n] += 2*np.pi
    cameraYawRange = np.sort(cameraYawRange)[::-1]

    return cameraYawRange, cameraYaw

### Remove voxels in view of the camera from the base grid

In [24]:
def in1d_dot_approach(A,B):
    """Returns the first array with elements from the second array removed"""
    cumdims = (np.maximum(A.max(),B.max())+1)**np.arange(B.shape[1])
    return A[~np.in1d(A.dot(cumdims),B.dot(cumdims))]

def removeVoxelsInView(baseVoxelGrid, voxelsInRange, \
                                    yawToVoxelsInRange, cameraYawRange):
    """Remove voxels that are in range, and in view of the camera"""
    if cameraYawRange[0]>np.pi/2 and cameraYawRange[1]<-np.pi/2:
        voxelsToRemove = voxelsInRange[np.logical_and(\
            yawToVoxelsInRange[:]>cameraYawRange[0], \
            yawToVoxelsInRange[:]<cameraYawRange[1]
            )]
    else:
        voxelsToRemove = voxelsInRange[np.logical_and(\
            yawToVoxelsInRange[:]<cameraYawRange[0], \
            yawToVoxelsInRange[:]>cameraYawRange[1]
            )]

    if voxelsToRemove.shape[0]!=0:
        voxelRemovedGrid = in1d_dot_approach(baseVoxelGrid, voxelsToRemove)
        return voxelRemovedGrid

    else:
        return baseVoxelGrid

### Combine unique voxels from base and new voxel grids

In [25]:
def combineVoxelGrids(baseVoxelGrid, newVoxelGrid):
    """Combine unique voxels from given voxel grids"""
    return np.unique(np.vstack([baseVoxelGrid, newVoxelGrid]), axis=0)

## Combined voxelizing function

In [26]:
def getNewVoxelGrid(disparityMapL, dispToDepthMatrix, rotationMatrix, \
        cameraPosition, voxelSize, occupancyThreshold, voxelStopFraction):
    """Generate a new voxel grid from the given disparity map"""
    # Compute point cloud
    pointCloud = generatePointCloud(disparityMapL, dispToDepthMatrix)
    # Filter point cloud
    pointCloud = filterPointCloud(pointCloud)
    # Compute new voxel grid
    newVoxelGrid, iterations = voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction)
    # Rotate voxel grid
    newVoxelGrid = rotateGrid(newVoxelGrid, rotationMatrix)
    # Translate voxel grid
    newVoxelGrid += cameraPosition

    return newVoxelGrid, iterations

### Testing combined function

In [27]:
cameraPosition = [0,0,0]
newVoxelGrid, iterations = getNewVoxelGrid(stereoMatcher.disparityMapL,\
                    imageProcessor.dispToDepthMatrix, redefineRotationMatrix, \
                    cameraPosition, voxelSize, occupancyThreshold, \
                    voxelStopFraction)

plotGrid(newVoxelGrid, voxelSize)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Combined pipeline

In [28]:
# Deques support thread-safe, memory efficient appends and pops from 
# either side with approximately the same O(1) performance in either 
# direction
# Pronounced 'deck'; 'double ended queue'
keypoints = deque(maxlen=2)
descriptors = deque(maxlen=2)

# Initializing empty voxel grid
baseVoxelGrid = None

# Initializing camera position, trajectory, and rotation
cameraPosition = np.array([0,0,0])
rotation = deque([redefineRotationMatrix.copy()])
trajectory = deque([np.array([0, 0, 0])])

R = redefineRotationMatrix.copy()
T = np.zeros([3, 1])
RT = np.hstack([R, T])
RT = np.vstack([RT, np.zeros([1, 4])])
RT[-1, -1] = 1

for imageL, imageR in zip(imagesL, imagesR):
    # Extract features and append to deques
    imageKeypoints, imageDescriptors = extractFeatures(imageL)
    keypoints.append(imageKeypoints)
    descriptors.append(imageDescriptors)

    # Estimate motion after the first frame/iteration
    if len(keypoints)==2:
        # Match features
        match = matchFeatures(descriptors[0], descriptors[1])

        # Estimate motion between the images
        rmat, tvec, imagePoints0, imagePoints1 = estimateMotion(match, \
                        keypoints[0], keypoints[1])

        # Estimate cumulative trajectory and rotation
        rt_mtx = np.hstack([rmat, tvec])
        rt_mtx = np.vstack([rt_mtx, np.zeros([1, 4])])
        rt_mtx[-1, -1] = 1

        rt_mtx_inv = np.linalg.inv(rt_mtx)
        
        RT = np.dot(RT, rt_mtx_inv)

        newTrajectory = RT[:3, 3]
        newRotation = RT[:3, :3]

        # Append updated trajectory and rotation matrices
        trajectory.appendleft(newTrajectory*100)
        rotation.appendleft(newRotation)

    # Convert image to grayscale and undistort
    imageProcessor.convertToGrayscale(imageL, imageR)
    imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

    # Compute disparity
    stereoMatcher.computeDisparity(\
                    grayImageL=imageProcessor.undistortImageL, \
                    grayImageR=imageProcessor.undistortImageR)
    stereoMatcher.clampDisparity()
    stereoMatcher.applyClosingFilter()

    # Compute new voxel grid
    newVoxelGrid, iterations = getNewVoxelGrid(stereoMatcher.disparityMapL,\
                    imageProcessor.dispToDepthMatrix, rotation[0], \
                    trajectory[0], voxelSize, occupancyThreshold, \
                    voxelStopFraction)
    
    # For all iterations except the first
    if baseVoxelGrid is not None:
        # Compute camera view range
        cameraYawRange, cameraYaw = getCameraYawRange(rotation[0])

        # Get voxels in range of the camera
        voxelsInRange, yawToVoxelsInRange, distanceToVoxelsInRange = \
                returnVoxelsInRange(baseVoxelGrid, trajectory[0])

        # Remove voxels from base grid in range and in view of the camera
        voxelRemovedGrid = removeVoxelsInView(baseVoxelGrid, voxelsInRange, \
                                    yawToVoxelsInRange, cameraYawRange)

        # Combine the new and base voxel grids
        baseVoxelGrid = combineVoxelGrids(voxelRemovedGrid, newVoxelGrid)

    # For the first iteration
    else:
        baseVoxelGrid = newVoxelGrid

In [33]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

# Plot accumulated voxel grid
ax.scatter(baseVoxelGrid[:,0], \
            baseVoxelGrid[:,1], \
            baseVoxelGrid[:,2], \
            s=voxelSize)

# Plot trajectory
trajectoryPlot = np.array(trajectory)
ax.plot(trajectoryPlot[:,0], \
        trajectoryPlot[:,1], \
        trajectoryPlot[:,2])

# Set axis labels
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Setting axis limits
# Camera axis begins at x=0 and looks to positive x
ax.set_xlim(0,2000)
ax.set_ylim(-1000,1000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …